<a href="https://colab.research.google.com/github/Indongspace/mulcamp34/blob/main/2%ED%9A%8C%EA%B8%B0%EC%B6%9C%EC%9C%A0%ED%98%95_%EC%97%B0%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 문제 1.
- 다음은 E-Commerce Shipping 데이터 세트이다. 주어진 훈련 데이터 세트를 활용하여 고객이 주문한 물품의 정시 도착 여부를 예측하고 해당 예측 결과를 csv 파일로 제출하시오.
- 결과 예측 양식 : 제출한 예측값의 ROC_AUC 점수 결과를 통해 영역별 배점에 따라 최종 점수가 반영될 예정

In [ ]:
import pandas as pd
pd.DataFrame({
    "ID"  : [0, 1, 2],
    "y_pred" : [0.2543, 0.1324, 0.5892]
})

,ID,y_pred
0,0,0.2543
1,1,0.1324
2,2,0.5892


## 데이터 불러오기

In [ ]:
import pandas as pd
X_test = pd.read_csv("data/2회/204_x_test.csv")
X_train = pd.read_csv("data/2회/204_x_train.csv")
y_train = pd.read_csv("data/2회/204_y_train.csv")

In [ ]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6599 entries, 0 to 6598
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   ID                   6599 non-null   int64
 1   Reached.on.Time_Y.N  6599 non-null   int64
dtypes: int64(2)
memory usage: 103.2 KB


## 데이터 정보

In [ ]:
import pandas as pd
pd.DataFrame({
    "변수" : list(X_train.columns) + ["Reached.on.Time_Y.N"],
    "설명" : [
        "고객 ID 번호",
        "창고의 블록 단위 구역(A, B, C, D, F)",
        "제품 배송 방법",
        "문의 전화 수",
        "고객의 등급(1: 가장 낮음, 5: 가장 높음)",
        "제품의 비용(달러 기준)",
        "사전 구매 수량",
        "제품의 중요도",
        "성별(F: 여성, M: 남성)",
        "할인혜택",
        "그램 단위 무게",
        "정시 도착 여부(1: 도착하지 않음, 0: 정시 도착)"
    ]
})

,변수,설명
0,ID,고객 ID 번호
1,Warehouse_block,"창고의 블록 단위 구역(A, B, C, D, F)"
2,Mode_of_Shipment,제품 배송 방법
3,Customer_care_calls,문의 전화 수
4,Customer_rating,"고객의 등급(1: 가장 낮음, 5: 가장 높음)"
5,Cost_of_the_Product,제품의 비용(달러 기준)
6,Prior_purchases,사전 구매 수량
7,Product_importance,제품의 중요도
8,Gender,"성별(F: 여성, M: 남성)"
9,Discount_offered,할인혜택


In [ ]:
import pandas as pd
X_test = pd.read_csv("data/2회/204_x_test.csv")
X_train = pd.read_csv("data/2회/204_x_train.csv")
y_train = pd.read_csv("data/2회/204_y_train.csv")


# 코드 작성
# 데이터 확인
#print(X_train.head(1))
#print(y_train.head(2))

# 결측치 확인
#print(X_train.isnull().sum()) 결측치 없음
#print(X_test.isnull().sum()) 결측치 없음

# 불필요한 컬럼 삭제(ID : 패턴이 안 보이기 때문)
X_train_id = X_train.pop('ID') # X_train_id는 저장, X_train.pop('ID') # ID컬럼은 제거 된 채로 DataFrame 저장
X_test_id = X_test.pop('ID')
#print(y_train['Reached.on.Time_Y.N'].value_counts()) # 비율의 차이가 나오니까, stratify 옵션 사용해야 함

#print(X_train.describe()) # 피처 엔지니어링은 RobustScaler 사용하기로 결정

# pipeline 사용 시, 사용하고자 하는컬럼명을 추출하는 것이 중요
# 숫자형 컬럼 추출
import numpy as np
numbers_df = X_train.select_dtypes(include=np.number)
#numbers_df.head(1)

# 문자형 컬럼 추출
object_df = X_train.select_dtypes(exclude=np.number)
#print(object_df.head(1)) # OrdinalEncoder, One-Hot Encoder 분리

from sklearn.model_selection import train_test_split
X_tr,X_te,y_tr,y_te = train_test_split(X_train,y_train['Reached.on.Time_Y.N'],
                                       stratify=y_train['Reached.on.Time_Y.N'],
                                       random_state=42,
                                      test_size=0.25)
print(X_tr.shape,X_te.shape,y_te.shape,y_te.shape)

(4949, 10) (1650, 10) (1650,) (1650,)


In [ ]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, RobustScaler, MinMaxScaler, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier
import numpy as np

# RandomizedSearchCV() 에 대입
param_grid = {
    'classifier__n_estimators' : np.arange(30, 1000, 100), # 필수
    'classifier__max_depth' : [None] + list(np.arange(5, 30, 5)), # 필수
    'classifier__min_samples_split': [2, 5, 10], # 옵션
    'classifier__min_samples_leaf': [1, 2, 4], # 옵션
    'classifier__max_features': ['auto', 'sqrt', 'log2'], # 옵션
}

# ColumnTransformer
column_transformer = ColumnTransformer([
    ("scaler", RobustScaler(), numbers_df.columns), # scaler
    ("ohc_encoder", OneHotEncoder(), ['Warehouse_block', 'Mode_of_Shipment', 'Gender']), # 명목척도 처리
    ("pd_imp_encoder", OrdinalEncoder(categories=[['low', 'medium', 'high']]), ['Product_importance']), # 서열척도 처리
], remainder = 'passthrough')

pipeline = Pipeline([
    ('preprocessor', column_transformer),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Grid Search 코드 스타일은 비슷(99% 비슷)
random_search = RandomizedSearchCV(
    estimator = pipeline,
    param_distributions = param_grid,
    n_iter = 3,
    scoring = 'roc_auc', # 잘 모르겠다. 시험장에서는 디폴트 값으로 accuracy
    cv = 3,
    verbose=3,
    n_jobs = -1 # CPU cores 모두 사용하겠다는 뜻
)

# 모형학습
random_search.fit(X_tr, y_tr)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(remainder='passthrough',
                                                                transformers=[('scaler',
                                                                               RobustScaler(),
                                                                               Index(['Customer_care_calls', 'Customer_rating', 'Cost_of_the_Product',
       'Prior_purchases', 'Discount_offered', 'Weight_in_gms'],
      dtype='object')),
                                                                              ('ohc_encoder',
                                                                               OneHotEncoder(),
                                                                               ['Warehouse_block',
                                                                                'Mode_of_Ship...
                                              RandomForestClassifier(random_state=42))]),
                   n_iter=3, n_jobs=-1,
                   param_distributions={'classifier__max_depth': [None, 5, 10,
                                                                  15, 20, 25],
                                        'classifier__max_features': ['auto',
                                                                     'sqrt',
                                                                     'log2'],
                                        'classifier__min_samples_leaf': [1, 2,
                                                                         4],
                                        'classifier__min_samples_split': [2, 5,
                                                                          10],
                                        'classifier__n_estimators': array([ 30, 130, 230, 330, 430, 530, 630, 730, 830, 930])},
                   scoring='roc_auc', verbose=3)

In [ ]:
# 예측값 산출
predictions = random_search.predict_proba(X_te)
# 모형평가
roc_auc_score(y_te,predictions[:,1])

0.7421350330140071

In [ ]:
final_preds = random_search.predict_proba(X_test)
result = pd.DataFrame({"ID" : X_test_id, "pred": final_preds[:, 1]})
print(result.head(1))
result.to_csv("00000.csv", index=False)

     ID      pred
0  7007  0.449037
